# Creditworthiness Investigation – Analysis of Banking Data

Research **Objective** - Test four hypotheses:

1) The number of children a borrower has impacts their loan repayment.
2) The marital status of a borrower affects their loan repayment.
3) The income level of a borrower influences their loan repayment.
4) Loan purposes have an impact on timely loan repayment.


The research will be conducted in three **stages:**

- Data overview
- Data preprocessing
- Hypothesis testing

## Data overview

In [1]:
# Import the pandas library
import pandas as pd

In [2]:
# Read the data file and store it in the variable data
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
# Get the first 10 rows of the data
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# Get general information about the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


The table consists of 12 columns.The number of values in the columns varies, which indicates that there are missing values in the data.

## Data Preprocessing

### Missing Values Removal

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In two columns, there are missing values. One of them is 'days_employed'. The missing values in this column will be addressed in the next step. The other column with missing values is 'total_income', which holds income data. Income amount is primarily influenced by employment type, so the missing values in this column will be filled with the median value for each income_type category from the 'income_type' column. For example, for an individual with an employment type of 'employee', the missing value in the 'total_income' column should be filled with the median income among all records with the same employment type.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Handling Anomalous Values

Let's display the median value of days_employed for each 'income_type', which represents the work experience, in days.

In [7]:
data['days_employed'] = data['days_employed'].abs()
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

For two types (unemployed and retirees), extremely large values will arise. We leave them as they are Moreover, as this column will not be needed for analysis.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In the 'children' column, there are two anomalous values. Let's remove these rows.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Missing Values Removal (Continuation)

Let's fill in the missing values in the column 'days_employed' with the median values for each 'income_type'.

In [10]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()
    
data.isna().sum()    

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Data Type Conversion

In [11]:
data['total_income'] = data['total_income'].astype(int)

### Duplicate Data Handling

In [12]:
data['education'] = data['education'].str.lower()

In [13]:
data.duplicated().sum()

71

In [14]:
data = data.drop_duplicates()

### Data Categorization

Based on the ranges provided below, we will create a column named "total_income_category" in the dataframe with the following categories:

- 0–30000: 'E'
- 30001–50000: 'D'
- 50001–200000: 'C'
- 200001–1000000: 'B'
- 1000001 and above: 'A'.

In [15]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass
    
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [16]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Let's create a function that, based on the data from the "purpose" column, will generate a new column called "purpose_category" with the following categories:

- 'Car-related operations'
- 'Real estate operations'
- 'Wedding expenses'
- 'Education expenses'.

In [17]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'
    
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Hypothesis Testing

#### Is there a correlation between the number of children and loan repayment?

In [18]:
# Group the data by the 'children' column, return the 'debt' values, and sum the number of debts
children_debt = data.groupby('children')['debt'].sum()  
children_debt

children
0    1063
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64

In [19]:
# Perform verification using pivot tables
data_children_debt_pivot = data.pivot_table(columns='children', values='debt', aggfunc='sum')  
data_children_debt_pivot

children,0,1,2,3,4,5
debt,1063,444,194,27,4,0


In [20]:
# Determine the distribution of borrowers based on the number of children they have
children_amount = data.groupby('children')['children'].count()  
children_amount

children
0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [21]:
# Calculate the debt ratio among each category and sort values in ascending order
child_debt_ratio = ((children_debt / children_amount) * 100).sort_values()  
child_debt_ratio

children
5    0.000000
0    7.543822
3    8.181818
1    9.234609
2    9.454191
4    9.756098
dtype: float64

Based on the available data, we cannot conclude that families with children have significantly different delays in loan repayments compared to families without children. The debt ratios are quite close, and slight changes in the sample could lead to different results. In other words, there isn't enough substantial difference between the groups to draw any meaningful conclusions about their significance.

#### Is there a correlation between marital status and loan repayment?

In [22]:
# Group the data by the 'family_status' column,
# return the 'debt' values, sum the number of debts,
# and sort in ascending order

family_status_debt = data.groupby('family_status')['debt'].sum().sort_values()  
family_status_debt

family_status
вдовец / вдова            63
в разводе                 84
Не женат / не замужем    273
гражданский брак         385
женат / замужем          927
Name: debt, dtype: int64

In [23]:
# Determine the distribution of borrowers based on marital status categories
family_amount = data.groupby('family_status')['family_status'].count().sort_values()  
family_amount

family_status
вдовец / вдова             951
в разводе                 1189
Не женат / не замужем     2796
гражданский брак          4134
женат / замужем          12261
Name: family_status, dtype: int64

In [24]:
# Calculate the debt ratio among each marital status category
family_ratio = (family_status_debt / family_amount) * 100  
family_ratio

family_status
вдовец / вдова           6.624606
в разводе                7.064760
Не женат / не замужем    9.763948
гражданский брак         9.313014
женат / замужем          7.560558
dtype: float64

The obtained values indicate that the lowest risk of not repaying a loan on time is 6.6% within the "widowed" category. The highest risk is 9.8% among borrowers categorized as "Single". A similar value (9.3%) is observed among borrowers in the "civil partnership" category. Furthermore, approximately equal values (around 7%) correspond to borrowers in the "divorced" and "married" categories. Therefore, there is no direct correlation between marital status and timely loan repayment.

#### Is there a correlation between income level and loan repayment?

In [25]:
# Group the data by the 'total_income_category' column,
# return the 'debt' values, sum the number of debts,
# and sort based on the index value

total_income_category_debt = data.groupby('total_income_category')['debt'].sum().sort_index()  
total_income_category_debt

total_income_category
A       2
B     354
C    1353
D      21
E       2
Name: debt, dtype: int64

In [26]:
# Calculate the count of borrowers based on income level categories
income_amount = data.groupby('total_income_category')['total_income_category'].count().sort_index()  
income_amount

total_income_category
A       25
B     5014
C    15921
D      349
E       22
Name: total_income_category, dtype: int64

In [27]:
# Calculate the debt ratio among each income level category
total_income_category_ratio = (total_income_category_debt / income_amount) * 100  
total_income_category_ratio

total_income_category
A    8.000000
B    7.060231
C    8.498210
D    6.017192
E    9.090909
dtype: float64

Based on common sense, it's logical to assume that individuals with lower incomes are more likely to become debtors, as financial difficulties might prevent them from making timely payments. However, when looking at our results, the conclusion is quite different: the sample of borrowers, both with the lowest and the highest incomes, is insufficient to draw definitive conclusions. We might consider that the E-category, with the lowest income, might struggle to manage their finances and therefore enter into credit agreements. However, it remains unclear why the A-category also has a relatively high debt percentage. In general, the conclusions regarding groups A and E are not clear due to the limited amount of data.

If we focus our analysis on groups B, C, and D, there is no observable correlation suggesting that "those who receive higher salaries are more likely to repay loans." This makes sense, as financial well-being isn't the sole factor influencing credit history quality.

#### How do different loan purposes affect their repayment?

In [28]:
# Group the data by the 'purpose_category' column,
# return the 'debt' values, sum the number of debts,
# and sort based on the index value

purpose_category_debt = data.groupby('purpose_category')['debt'].sum().sort_index()  
purpose_category_debt

purpose_category
операции с автомобилем      400
операции с недвижимостью    780
получение образования       369
проведение свадьбы          183
Name: debt, dtype: int64

In [29]:
# Calculate the count of borrowers based on loan purpose categories
purpose_category_amount = data.groupby('purpose_category')['purpose_category'].count().sort_index()  
purpose_category_amount


purpose_category
операции с автомобилем       4279
операции с недвижимостью    10751
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

In [30]:
# Calculate the debt ratio among each loan purpose category
purpose_category_ratio = (purpose_category_debt / purpose_category_amount) * 100  
purpose_category_ratio

purpose_category
операции с автомобилем      9.347978
операции с недвижимостью    7.255139
получение образования       9.252758
проведение свадьбы          7.911803
dtype: float64

The obtained values indicate that the risk of not repaying a loan on time varies within the range of 7-9%. There is no direct correlation between the purpose of the loan and the presence of debt.
Absolutely, the findings align with both the numbers and common sense. People tend to buy cars on credit that are much more expensive than they can afford based on their income (who doesn't dream of a nice car?). On the other hand, individuals are more cautious about risking their mortgage payments. It's interesting that education loans don't follow the same trend – typically, those who take out such loans are motivated individuals with specific goals ahead of them. Moreover, the amounts for such loans, given the online education market, are not as substantial.

## Conclusion

An investigation of the bank's credit department data was conducted to determine whether marital status and the number of children influence the timely repayment of loans. The research findings will be used in building a credit scoring model, a specialized system that assesses a potential borrower's ability to repay a loan to the bank. The input data from the bank includes statistics on clients' creditworthiness.

<b>Research Hypotheses:</b>
1) The number of children a borrower has impacts their loan repayment.
2) The marital status of a borrower affects their loan repayment.
3) The income level of a borrower influences their loan repayment.
4) Loan purposes have an impact on timely loan repayment.

Based on the research results:

<b>Is there a correlation between the number of children and timely loan repayment?</b>

children:

5    0.000000

0    7.543822

3    8.181818

1    9.234609

2    9.454191

4    9.756098

The obtained values show that the distribution of debtors among borrowers with 1 to 4 children is approximately the same: around 9% of borrowers in these categories did not repay the loan on time. The risk is slightly lower for childless borrowers, with 7% having debt. Borrowers with 5 children have the lowest risk of payment delays.

<b>Is there a correlation between marital status and timely loan repayment?</b>

family_status:

widow           6.624606

divorced        7.064760

Single          9.763948

civil partnership         9.313014

married          7.560558

The obtained values indicate that the lowest risk of not repaying a loan on time is 6.6% in the "widow" category. The highest risk is 9.8% among borrowers categorized as "Single". Similar values (around 9.3%) are observed for borrowers in the "civil partnership" category. Additionally, roughly equal values (around 7%) correspond to borrowers in the "divorced" and "married" categories. Therefore, no direct correlation between marital status and timely loan repayment was identified.

<b>Is there a correlation between income level and timely loan repayment?</b>

total_income_category:

A    8.000000

B    7.060231

C    8.498210

D    6.017192

E    9.090909

Income level distribution by categories:

0–30 000 — 'E';

30 001–50 000 — 'D';

50 001–200 000 — 'C';

200001–1 000 000 — 'B';

Above 1 000 001 — 'A'.

The obtained values indicate that the risk of not repaying a loan on time varies within the range of 6-9%, but there is no direct correlation between income level and debt.

<b>How do different loan purposes affect their timely repayment?</b>

purpose_category:

car purchases      9.347978

real estate transactions      7.255139

education expenses         9.252758

wedding expenses         7.911803

The obtained values indicate that the risk of not repaying a loan on time varies within the range of 7-9%, and there is no direct correlation between loan purpose and debt.

<b>Overall Conclusion and Recommendations:</b>

1) Based on the findings, it can be concluded that there is no direct correlation between the examined categories (number of children, marital status, income level, loan purposes) and timely loan repayment. On average, the risk of non-repayment (or repayment not on time) ranges from 6-10%.

2) It is worth checking whether the combination of the examined categories has an impact on the likelihood of timely loan repayment or considering other categories.

3) Gathering additional data for these categories is necessary for further hypothesis testing.